In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 100 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 11
piter = 5
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_CO2 = 0.0001

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}

In [4]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}

In [5]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [6]:
#Read-in deterministic data observations for use in inference.
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', 4, t, 0.1)
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', 5, t, 0.1)
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', 5, t, 0.1)

In [7]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [8]:
obs_model_CON_no_CO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_CON[:-1, :], SCALE = obs_error_CON[:, :-1])
obs_model_AWB_no_CO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB[:-1, :], SCALE = obs_error_AWB[:, :-1])
obs_model_AWB_ECA_no_CO2 = ObsModel(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB_ECA[:-1, :], SCALE = obs_error_AWB_ECA[:, :-1])

In [9]:
obs_model_CON_CO2 = ObsModelCO2(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_CON, SCALE = obs_error_CON, GET_CO2 = get_CO2_CON)
obs_model_AWB_CO2 = ObsModelCO2(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB, SCALE = obs_error_AWB, GET_CO2 = get_CO2_AWB)
obs_model_AWB_ECA_CO2 = ObsModelCO2(DEVICE = devi, TIMES = obs_times, DT = dt, MU = obs_means_AWB_ECA, SCALE = obs_error_AWB_ECA, GET_CO2 = get_CO2_AWB)

In [10]:
net = SDEFlow(devi, batch_size, obs_model_CON_no_CO2, state_dim_SCON, t, dt, n).to(devi)
C_PATH, log_prob = net()
print('C_PATH:', C_PATH)
print('C_PATH.size():', C_PATH.size())

C_PATH: tensor([[[0.7721, 0.5603, 0.5303],
         [0.9073, 0.4425, 0.5224],
         [0.8167, 0.6699, 0.6588],
         ...,
         [1.3747, 0.7573, 0.6534],
         [1.5648, 0.7414, 0.6495],
         [1.3807, 0.6547, 0.5766]],

        [[0.5764, 1.6628, 0.7585],
         [0.9958, 0.7663, 0.7310],
         [2.4154, 0.4168, 0.7498],
         ...,
         [0.7133, 0.7177, 0.6348],
         [0.7804, 0.7253, 0.5036],
         [0.4995, 0.7242, 0.5204]]], grad_fn=<AddBackward0>)
C_PATH.size(): torch.Size([2, 500, 3])


In [11]:
test_CO2 = get_CO2_CON(C_PATH, t_span_tensor[:, 1:, :], SCON_C_params_dict, temp_gen, temp_ref)
print(test_CO2)
torch.cat([C_PATH, test_CO2], -1)

tensor([[[0.0020],
         [0.0017],
         [0.0026],
         [0.0032],
         [0.0025],
         [0.0032],
         [0.0035],
         [0.0027],
         [0.0030],
         [0.0036],
         [0.0038],
         [0.0044],
         [0.0037],
         [0.0031],
         [0.0036],
         [0.0040],
         [0.0051],
         [0.0054],
         [0.0048],
         [0.0039],
         [0.0040],
         [0.0035],
         [0.0050],
         [0.0058],
         [0.0044],
         [0.0027],
         [0.0045],
         [0.0061],
         [0.0040],
         [0.0032],
         [0.0038],
         [0.0048],
         [0.0032],
         [0.0033],
         [0.0044],
         [0.0040],
         [0.0044],
         [0.0058],
         [0.0036],
         [0.0037],
         [0.0043],
         [0.0040],
         [0.0033],
         [0.0032],
         [0.0045],
         [0.0040],
         [0.0034],
         [0.0041],
         [0.0037],
         [0.0023],
         [0.0026],
         [0.0027],
         [0.

tensor([[[7.7209e-01, 5.6026e-01, 5.3034e-01, 2.0398e-03],
         [9.0726e-01, 4.4252e-01, 5.2244e-01, 1.6962e-03],
         [8.1666e-01, 6.6988e-01, 6.5879e-01, 2.6386e-03],
         ...,
         [1.3747e+00, 7.5731e-01, 6.5338e-01, 5.0436e-03],
         [1.5648e+00, 7.4142e-01, 6.4950e-01, 5.0558e-03],
         [1.3807e+00, 6.5475e-01, 5.7656e-01, 4.5534e-03]],

        [[5.7644e-01, 1.6628e+00, 7.5846e-01, 5.9098e-03],
         [9.9583e-01, 7.6628e-01, 7.3097e-01, 2.9012e-03],
         [2.4154e+00, 4.1680e-01, 7.4975e-01, 1.7310e-03],
         ...,
         [7.1330e-01, 7.1771e-01, 6.3478e-01, 4.7580e-03],
         [7.8036e-01, 7.2531e-01, 5.0361e-01, 4.8777e-03],
         [4.9948e-01, 7.2420e-01, 5.2042e-01, 4.9575e-03]]],
       grad_fn=<CatBackward>)

In [12]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, OBS_MODEL_NO_CO2, OBS_MODEL_CO2, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    net = SDEFlow(DEVICE, BATCH_SIZE, OBS_MODEL_NO_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= PRETRAIN_ITER:
                CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                x_with_CO2 = torch.cat([C_PATH, CO2], -1)
                l1_norm_element = x_with_CO2 - torch.mean(OBS_MODEL_CO2.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(OBS_MODEL.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx mean =', x_with_CO2.mean(-2))
                    print('\nx =', x_with_CO2)
                l1_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - OBS_MODEL_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\n C_PATH =', C_PATH)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [13]:
train(devi, l_r, niter, piter, batch_size, obs_model_CON_no_CO2, obs_model_CON_CO2, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000004574.079687. Best norm loss value is: 45740.796875.

x mean = tensor([[0.9430, 0.8013, 0.7671, 0.0033],
        [0.9302, 0.8179, 0.7729, 0.0033]], grad_fn=<MeanBackward1>)

x = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [1.0113e+00, 1.0735e+00, 6.4779e-01, 3.8488e-03],
         [5.4000e-01, 6.4772e-01, 5.5459e-01, 2.4374e-03],
         ...,
         [6.4111e-01, 5.3357e-01, 6.7643e-01, 3.5941e-03],
         [6.3734e-01, 8.2016e-01, 6.0289e-01, 5.5132e-03],
         [4.2456e-01, 1.3621e+00, 6.2565e-01, 9.2069e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.5354e-01, 7.0178e-01, 1.6096e+00, 2.6777e-03],
         [6.6290e-01, 9.8644e-01, 5.8204e-01, 3.6710e-03],
         ...,
         [4.5756e-01, 1.1055e+00, 6.9147e-01, 7.2277e-03],
         [7.1542e-01, 1.0504e+00, 7.0893e-01, 7.0398e-03],
         [6.6767e-01, 7.4443e-01, 1.2377e+00, 5.2893e-03]]],
       grad_fn=<CatBackward>)



Train Diffusion:  55%|█████▍    | 6/11 [00:02<00:02,  2.13it/s]


RuntimeError: The size of tensor a (501) must match the size of tensor b (11) at non-singleton dimension 1